# Emotion Prediction from a Given Picture Using FER2013 Dataset

# Importing Libraries & Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras import backend as k

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import cv2

In [ ]:
df=pd.read_csv('icml_face_data.csv')

df.head()

In [ ]:
df.columns=['emotion','Usage','pixels']

In [ ]:
#df['pixels'][0]

In [ ]:
def string2array(x):
    return np.array(x.split(' ')).reshape(48,48).astype('float32')
df['pixels']=df['pixels'].apply(lambda x: string2array(x))

In [ ]:
df

In [ ]:
# shuffle the dataframe's rows
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.emotion.value_counts().index

In [ ]:
label_to_text={0: 'anger', 1: 'disgust', 2:'fear', 3:'happiness', 4: 'sad', 5: 'surprised', 6:'neutral'}

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(hue=[label_to_text[i] for i in df.emotion.value_counts().index], x=[label_to_text[i] for i in df.emotion.value_counts().index],
           y=df.emotion.value_counts(), palette='pastel',legend=False)

In [ ]:
df.emotion.unique()

In [ ]:
#showing pictures of training set with corresponding labels
label_to_text={0: 'anger', 1: 'disgust', 2:'fear', 3:'happiness', 4: 'sad', 5: 'surprised', 6:'neutral'}
count=0
fig,axs=plt.subplots(7,7,figsize=(16,16))
for i in df.emotion.unique():
    data=df[df['emotion']==i]
    for img in data['pixels']:
        axs[i][count].imshow(img,cmap='gray')
        axs[i][count].title.set_fontsize(14)
        axs[i][count].title.set_fontweight('bold')
        axs[i][count].title.set_color('red')
        axs[i][count].title.set_backgroundcolor('yellow')
        axs[i][count].title.set_text(label_to_text[i])
        count+=1
        if count==7:
           break;
    count=0
    fig.tight_layout()


# Data Preparation & Image Augmentation

In [ ]:
#split the dataframe into features and labels
from tensorflow.keras.utils import to_categorical

In [ ]:
X=df['pixels']
y=to_categorical(df['emotion'])
X.size

In [ ]:
X.shape[0]

In [ ]:
type(X)

In [ ]:
np.stack(X,axis=0).shape

In [ ]:
np.stack(X,axis=0).shape
X=np.stack(X,axis=0)
X=X.reshape(df.shape[0],48,48,1)
X.shape,y.shape

In [ ]:
X.shape

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.15,random_state=42)
X_test,X_val,y_test,y_val=train_test_split(X_val,y_val,test_size=0.15,random_state=42)

In [ ]:
print('Train Set:', X_train.shape,y_train.shape)
print('Val Set:', X_val.shape,y_val.shape)
print('Test Set:', X_test.shape,y_test.shape)

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_set=train_datagen.flow(X_train,y_train,batch_size=64)
val_test_datagen=ImageDataGenerator(rescale=1./255)
val_set=val_test_datagen.flow(X_val,y_val,batch_size=64)
test_set=val_test_datagen.flow(X_test,y_test,batch_size=64)

# Build and Train Deep Learning Model for Facial Expression Classification

In [ ]:
def res_block(X,filter):
    #convolutional block
    X_copy=X
    f1,f2,f3=filter
    #Main path
    X=Conv2D(f1,(1,1),strides=(1,1))(X)
    X=MaxPool2D((2,2))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=Conv2D(f2,kernel_size=(3,3),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=Conv2D(f3,kernel_size=(1,1),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)

    #Short path
    X_copy=Conv2D(f3,kernel_size=(1,1),strides=(1,1))(X_copy)
    X_copy=MaxPool2D((2,2))( X_copy)
    X_copy=BatchNormalization(axis=3)(X_copy)
    #Add
    X=Add()([X,X_copy])
    X=Activation('relu')(X)
    return X


In [ ]:
def identity_block(X,filter):
    X_copy=X
    f1,f2,f3=filter
    #Main path
    X=Conv2D(f1,(1,1),strides=(1,1))(X)
    #X=MaxPool2D((2,2))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=Conv2D(f2,kernel_size=(3,3),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    X=Conv2D(f3,kernel_size=(1,1),strides=(1,1),padding='same')(X)
    X=BatchNormalization(axis=3)(X)
    #Add
    #X=Add()([X,X_copy])
    X=Activation('relu')(X)
    return X

In [ ]:
def get_model():
    input_shape=(48,48,1)
    #input tensor shape
    X_input=Input(input_shape)
    #1 Stage
    X=Conv2D(64,(3,3),padding='same')(X_input)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)

    #2 Stage
    X=res_block(X,filter=[64,128,32])
    X=identity_block(X,filter=[64,128,32])

    #3 stage
    X=res_block(X,filter=[128,256,64])
    X=identity_block(X,filter=[128,256,64])

    #4 stage
    X=res_block(X,filter=[256,512,128])
    X=identity_block(X,filter=[256,512,128])

    #Average Pooling
    X=GlobalAveragePooling2D()(X)

    #Final layer
    X=Flatten()(X)
    X=Dense(512,activation='relu')(X)
    X=Dense(7,activation='softmax')(X)
    model=Model(inputs=X_input,outputs=X)
    model.summary()
    return model


In [ ]:
input_shape=(48,48,1)
X_input=Input(input_shape)
X_input

In [ ]:
 X=Conv2D(64,(3,3),padding='same')(X_input)
 X=BatchNormalization(axis=3)(X)
 X=Activation('relu')(X)
 X

In [ ]:
X=res_block(X,filter=[64,128,32])
X=identity_block(X,filter=[64,128,32])
X

In [ ]:
facial_model=get_model()

In [ ]:
#!pip install pydot
#!pip install graphviz --upgrade

In [ ]:
plot_model(facial_model,show_shapes=True)

In [ ]:
#!pip install pydot


In [ ]:
#compile the network
#facial_model.compile(optimizer='adamax',loss='categorical_crossentropy',metrics=['accuracy'])

#define callback functions
#earlystopping=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=30)
#path='Facial_expression_weights.keras'
#checkpointer=ModelCheckpoint(filepath=path,verbose=1,save_best_only=True)
#reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=15,verbose=1,mode='min')

In [ ]:
#compile the network
facial_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#define callback functions
earlystopping=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=20)
path='Facial_expression_weights.keras'
checkpointer=ModelCheckpoint(filepath=path,verbose=1,save_best_only=True)
reduce_lr=ReduceLROnPlateau(monitor='val_loss',factor=0.001,patience=3,verbose=1,mode='min')

In [ ]:
#Uncomment below for retraining.
#h=facial_model.fit(train_set,validation_data=val_set,epochs=60,callbacks=[checkpointer,earlystopping,reduce_lr])

In [ ]:
facial_model.load_weights(path)

In [ ]:
_,acc=facial_model.evaluate(test_set)
print("Accuracy on test set {:.2f}%".format(acc*100))

In [ ]:
#Uncomment below for retraining.
#plt.figure(figsize=(12,6))
#plt.subplot(1,2,1)
#plt.plot(h.history['loss'])
#plt.plot(h.history['val_loss'])
#plt.title('Loss vs. Epoch')
#plt.ylabel('Loss')
#plt.xlabel('Epoch')
#plt.legend(['train','eval'])
#plt.subplot(1,2,2)
#plt.plot(h.history['accuracy'])
#plt.plot(h.history['val_accuracy'])
#plt.title('Accuracy vs Epoch')
#plt.ylabel('Acc')
#plt.xlabel('Epoch')
#plt.legend(['train','eval'])
#plt.show()

# Getting Prediction from a Given Picture

In [ ]:
#Getting the picture. For better prediction, the face must be cropped.
import tkinter
from tkinter import filedialog
file=tkinter.filedialog.askopenfilenames()


In [ ]:
image2=cv2.imread(file[0])
copy=image2.copy()
copy=cv2.cvtColor(copy,cv2.COLOR_BGR2RGB)
image2=cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)
image2=cv2.resize(image2,(48,48))


In [ ]:
image2=image2/255.
img_array = keras.utils.img_to_array(image2)
img_array = keras.ops.expand_dims(img_array,0) 
pred=facial_model.predict(img_array)


In [ ]:
import operator
max=0
for i in range(0,6):
    if operator.gt(pred[0,i],max):
        max=pred[0,i]
        idx=i


# Showing the Loaded Picture and the Predicted Emotion

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_title('Prediciton={}\n'.format(label_to_text[idx]))
ax.title.set_backgroundcolor('yellow')
ax.title.set_color('blue')
ax.title.set_fontweight('bold')
plt.imshow(copy)
plt.show()